In [6]:
import os
import spotipy
import spotipy.util as util
import time
import csv
import pandas as pd
import sys
import json
import csv

In [35]:
scope = 'user-library-read'
username = 'your_spotify_username'
token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
print(token)
sp = spotipy.Spotify(auth=token)

BQD3jywH5qLmnUQPPyaeKNaWu2Ly1G13df7qh7K_7W9-UK7Dj-3UCc04mXFblziwFSS6Uqrp3-OeAFp72x7YQurv3yB2P1Tbj7qQoc_iDGGK90TSD3U8UIA7_p7VQfJ0yvkI2SA15-OiNbbaCRYBGDwBkTiEP0U


In [8]:
def get_playlists_from_top_categories(sp, categories = None,limit = 50):
    playlist_ids = set()
    if categories == None:
        categories = sp.categories(limit=limit)
        for category in categories['categories']['items']:
            print(category['id'])
            try:
                category_playlists = sp.category_playlists(category['id'], limit=limit)
                for playlist in category_playlists['playlists']['items']:
                    playlist_ids.add(playlist['id'])
            except Exception as e:
                print(e)
                print("Could not get playlists for category ", category['id'])
    return playlist_ids

In [9]:
def get_playlists_from_specific_categories(sp, categories, limit = 50):
    playlist_ids = set()
    for category in categories:
        try:
            category_playlists = sp.category_playlists(category, limit=limit)
            for playlist in category_playlists['playlists']['items']:
                playlist_ids.add(playlist['id'])
        except Exception as e:
            print(e)
            print("Could not get playlists for category ", category['id'])
    return playlist_ids

In [10]:
playlist_ids = list(get_playlists_from_top_categories(sp))
#pop_playlist_ids = list(get_playlists_from_specific_categories(sp, ['pop', 'toplists']))
#print(pop_playlist_ids)

toplists
pop
hiphop
mood
workout
decades
country
focus
latin
chill
edm_dance
rnb
rock
indie_alt
roots
party
sleep
classical
jazz
inspirational
ellen
gaming
romance
kpop
popculture
arab
desi
afro
blackhistorymonth
metal
reggae
soul
blues
punk
funk
dinner
sessions
travel
family
comedy
word


In [14]:
print(len(playlist_ids))
from random import shuffle
shuffle(playlist_ids)

1312


In [16]:
def get_track_ids_from_playlists(sp, playlist_ids, last_playlist_index = 0, track_ids = set()):
    '''
    playlist_ids: List of playlist ids
    last_playlist_index: Only consider playlist_ids[last_playlist_index:]. Used to keep track
    of which playlist we are on in case we encounter an exception and need to call function again
    track_ids: Set that contains the current track ids we've obtained
    returns: List of song ids
    
    '''
    try:
        for idx, playlist_id in enumerate(playlist_ids[last_playlist_index:]):
            time.sleep(1)
            results = sp.user_playlist('spotify', playlist_id, fields='tracks,next,name')
            
            # Progress Indication and sleep for 10 seconds to not overload API
            if idx % 10 == 0:
                print("PLAYLIST ID IS: ", playlist_id)
                print(last_playlist_index/ len(playlist_ids), 'PERCENT DONE')
                print('Processed: ', len(track_ids), ' tracks')
                time.sleep(10)
            
            with open('all_song_ids.csv','a') as fd:
                csv_writer = csv.writer(fd)
                for track in results['tracks']['items']:
                    if track != None and track['track'] != None:
                        track_id = track['track']['id']
                        track_ids.add(track_id)
                        csv_writer.writerow([track_id])
            last_playlist_index += 1
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last playlist index was: ", last_playlist_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_track_ids_from_playlists(sp, playlist_ids, last_playlist_index, track_ids)

    return track_ids

In [17]:
track_ids = get_track_ids_from_playlists(sp, playlist_ids)
track_ids = list(track_ids)

PLAYLIST ID IS:  37i9dQZF1DX7G4CqN9FLiF
0.0 PERCENT DONE
Processed:  0  tracks
PLAYLIST ID IS:  37i9dQZF1DWSrj7tqQ9IOu
0.007621951219512195 PERCENT DONE
Processed:  648  tracks
PLAYLIST ID IS:  37i9dQZF1DX0Uv9tZ47pWo
0.01524390243902439 PERCENT DONE
Processed:  1197  tracks
PLAYLIST ID IS:  37i9dQZF1DWWzVPEmatsUB
0.022865853658536585 PERCENT DONE
Processed:  1811  tracks
PLAYLIST ID IS:  37i9dQZF1DWXDJDWnzE39E
0.03048780487804878 PERCENT DONE
Processed:  2258  tracks
PLAYLIST ID IS:  37i9dQZF1DXcNdPSCaTGoa
0.038109756097560975 PERCENT DONE
Processed:  2867  tracks
PLAYLIST ID IS:  37i9dQZF1DWYkaDif7Ztbp
0.04573170731707317 PERCENT DONE
Processed:  3524  tracks
PLAYLIST ID IS:  37i9dQZF1DX8a1tdzq5tbM
0.053353658536585365 PERCENT DONE
Processed:  4048  tracks
PLAYLIST ID IS:  37i9dQZF1DX4WgZiuR77Ef
0.06097560975609756 PERCENT DONE
Processed:  4620  tracks
PLAYLIST ID IS:  37i9dQZF1DXcBWIGoYBM5M
0.06859756097560976 PERCENT DONE
Processed:  5244  tracks
PLAYLIST ID IS:  37i9dQZF1DWXnexX7Ck

PLAYLIST ID IS:  37i9dQZF1DX24Kh15JFQJ0
0.6402439024390244 PERCENT DONE
Processed:  43232  tracks
PLAYLIST ID IS:  37i9dQZF1DX8QB9Ys2nV17
0.6478658536585366 PERCENT DONE
Processed:  43685  tracks
PLAYLIST ID IS:  37i9dQZF1DX9LT7r8qPxfa
0.6554878048780488 PERCENT DONE
Processed:  44078  tracks
PLAYLIST ID IS:  37i9dQZF1DX8MbMfAHb8U0
0.663109756097561 PERCENT DONE
Processed:  44584  tracks
PLAYLIST ID IS:  37i9dQZF1DWV1duDvis8iF
0.6707317073170732 PERCENT DONE
Processed:  45135  tracks
PLAYLIST ID IS:  37i9dQZF1DX8uVDy7wwKj9
0.6783536585365854 PERCENT DONE
Processed:  45548  tracks
PLAYLIST ID IS:  37i9dQZF1DWVlWpJblBvap
0.6859756097560976 PERCENT DONE
Processed:  45995  tracks
PLAYLIST ID IS:  37i9dQZF1DWVEcPtggpQMu
0.6935975609756098 PERCENT DONE
Processed:  46390  tracks
PLAYLIST ID IS:  37i9dQZF1DWT34oeYRnJ0R
0.7012195121951219 PERCENT DONE
Processed:  46644  tracks
PLAYLIST ID IS:  37i9dQZF1DX7oMO417tEZs
0.7088414634146342 PERCENT DONE
Processed:  47002  tracks
PLAYLIST ID IS:  37i9

In [18]:
print(len(track_ids))

63234


In [ ]:
def get_audio_features_and_analysis_from_song_id(sp, song_id):
    features = sp.audio_features(song_id)
    analysis = sp.audio_analysis(song_id)
    return (features, analysis)

In [19]:
features, analysis = get_audio_features_and_analysis_from_song_id(sp,'4kV4N9D1iKVxx1KLvtTpjS')
print(sys.getsizeof([features]))
# It takes 80 bytes to represent a song and its features/analysis...
# With 70k tracks, that's a 3.6GB dataset

72


In [20]:
def remove_string_keys_from_analysis(analysis):
    '''
    analysis: the dictionary returned from sp.audio_analysis(song_id)
    Remove the strings from the analysis because they take up a lot of space
    return: analysis but with said keys removed
    '''
    for key in ['codestring', 'echoprintstring', 'synchstring', 'rhythmstring']:
        analysis['track'].pop(key, None)
    return analysis

In [21]:
def get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, consecutive_errors = 0, song_features_and_analysis = {}):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        with open('incomplete_pop_track_id_features_and_analysis.json','w') as outfile:
            json.dump(song_features_and_analysis, outfile)
            
        return get_audio_features_and_analysis_from_songs(sp, track_ids, start_index + 1, end_index, 0, song_features_and_analysis)
    try:
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features, analysis = get_audio_features_and_analysis_from_song_id(sp, track_id)
            analysis = remove_string_keys_from_analysis(analysis)
            song_features_and_analysis[track_id] = [features, analysis]
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a .csv
            if idx % 50 == 0:
                print("SONG ID IS: ", track_id)
                print("Songs left: ", end_index - start_index)
                # time.sleep(1)
                #with open('pop_track_id_features_and_analysis.json','a') as outfile:
                #  json.dump(song_features_and_analysis, outfile)
                # song_features_and_analysis = {}
            start_index += 1
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, consecutive_errors + 1, song_features_and_analysis)
    with open('pop_track_id_features_and_analysis.json','a') as outfile:
        json.dump(song_features_and_analysis, outfile)
    return 'Success'

In [50]:
# get_audio_features_and_analysis_from_songs(sp, track_ids, 0, 2557)

SONG ID IS:  15O20RQyWJgKrkHID9ynT9
Songs left:  2557
SONG ID IS:  3dskm5fAe8OkBtM9E0VVWx
Songs left:  2507
SONG ID IS:  6ILpnOUHollfHp4xWH7nqV
Songs left:  2457
SONG ID IS:  6vPshkNpkYCwWIq9AQDtwt
Songs left:  2407
SONG ID IS:  2bcUvmxgEMfZD8QcEnZn7G
Songs left:  2357
SONG ID IS:  2oPCR02nduYVpj7CTfsA0d
Songs left:  2307
SONG ID IS:  51cCG7fjfxhi5jbu8ctVzl
Songs left:  2257
SONG ID IS:  5rFj8OzffT0iggSjcBaeND
Songs left:  2207
SONG ID IS:  3mD8rKs6rtOO3RUwNTIweI
Songs left:  2157
SONG ID IS:  1qCuOj9vIku6yHw2m3maPf
Songs left:  2107
SONG ID IS:  2pes8dcykE4OjmNkFstKI0
Songs left:  2057
SONG ID IS:  1ArLnG0bHaHPXqf08sEKM0
Songs left:  2007
SONG ID IS:  4vmgP10TH0YYqJkzr8yLW0
Songs left:  1957
SONG ID IS:  4R5ibiJ2duf2rUkjyLTXpB
Songs left:  1907
SONG ID IS:  4lJVxj98OFHoosOqZi78Ts
Songs left:  1857
SONG ID IS:  1pur8kv69XmZqA3uoMEYnH
Songs left:  1807
SONG ID IS:  1YNQscOx6OqBQjxgJVhEeW
Songs left:  1757
SONG ID IS:  5sdb5pMhcK44SSLsj1moUh
Songs left:  1707
SONG ID IS:  6GL6RkpLjE41nlz

'Success'

In [22]:
print('hi')

hi


In [21]:
with open('all_song_ids.csv', 'r') as f:
    reader = csv.reader(f)
    track_ids = list(reader)

In [27]:
track_ids = [item for sublist in track_ids for item in sublist]
track_ids = list(set(track_ids))


In [31]:
print(len(track_ids))

63234


In [33]:
with open('all_song_ids_no_dupes.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     for id in track_ids:
        wr.writerow([id])

In [ ]:
def get_audio_features_from_songs(sp, track_ids, start_index, end_index, consecutive_errors = 0, song_features = {}):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        with open('incomplete_all_track_id_features.json','w') as outfile:
            json.dump(song_features, outfile)
            
        return get_audio_features_from_songs(sp, track_ids, start_index + 1, end_index, 0, song_features)
    try:
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features, _ = get_audio_features_and_analysis_from_song_id(sp, track_id)
            song_features[track_id] = features
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a .csv
            if idx % 50 == 0:
                print("SONG ID IS: ", track_id)
                print("Songs left: ", end_index - start_index)
                # time.sleep(1)
                #with open('pop_track_id_features_and_analysis.json','a') as outfile:
                #  json.dump(song_features_and_analysis, outfile)
                # song_features_and_analysis = {}
            start_index += 1
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_audio_features_from_songs(sp, track_ids, start_index, end_index, consecutive_errors + 1, song_features)
    with open('all_track_id_features_0_5000.json','a') as outfile:
        json.dump(song_features, outfile)
    return 'Success'

In [ ]:
get_audio_features_from_songs(sp, track_ids, 0, 5000)

name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  0
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
AP

API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  3
Calling function again 
Too many consecutive errors.
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  

API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  13
Calling function ag

API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  21
Calling function again 
Too many consecutive errors.
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index

Last song index was:  31
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  31
Calling function again 
Too many consecutive errors.
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  32
Calling 

...

NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError:


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError:

NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-37-21f42af49701>", line 16, in get_audio_features_from_songs
NameError: name 'get_audio_features_and_analysis_from_song_id' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython

95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analysis_from_song_id' is not defined
API called failed
Last song index was:  95
Calling function again 
name 'get_audio_features_and_analys